<a href="https://colab.research.google.com/github/Svanzi/Tennis-Shot-Recognition/blob/main/Tennis%20Shot%20Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Svanzi/Tennis-Shoot-Recognition

Cloning into 'Tennis-Shoot-Recognition'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 10.37 KiB | 2.59 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [1]:
!python -m pip install mediapipe
!pip3 install tqdm

In [2]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import csv
from google.colab.patches import cv2_imshow
from tqdm.notebook import tqdm
from IPython.display import Video, display

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def write_landmarks_to_csv(landmarks, frame_number, csv_data):
    print(f"Landmark coordinates for frame {frame_number}:")
    for idx, landmark in enumerate(landmarks):
        print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
        csv_data.append([frame_number, mp_pose.PoseLandmark(idx).name, landmark.x, landmark.y, landmark.z])
    print("\n")

### Input & Output Paths


In [20]:
in_path = '/content/drive/MyDrive/University/Deep/Sinner_2min.mp4'
out_path = '/content/drive/MyDrive/University/Deep/Sinner_2min_annoted.mp4'
csv_path = '/content/drive/MyDrive/University/Deep/Sinner_2min_landmarks.csv'

### MediaPipe Setup

In [21]:
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

### Video Capture


In [22]:
# Open the video file
cap = cv2.VideoCapture(in_path)
if not cap.isOpened():
    print(f"Error opening video file: {in_path}")

fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

### Video Writer


In [23]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
if not writer.isOpened():
    print(f"Error opening video writer: {out_path}")

### CSV


In [24]:
write_csv = True
if write_csv:
    csv_file = open(csv_path, "w", newline="")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Frame", "Landmark", "Body Part" "x Pos", "y Pos", "z Pos", "Visibility"])

In [27]:
def dump_body_to_csv(landmarks, frame_idx, csv_writer):
    """Scrive solo i landmark corpo nel CSV."""
    for idx in BODY_IDX:
        lm = landmarks[idx]
        lm_name  = mp_pose.PoseLandmark(idx).name
        csv_writer.writerow([frame_idx, idx, lm_name, lm.x, lm.y, lm.z, getattr(lm, "visibility", 1.0)])

### Video Processing

Filtraggio dei landmarks interessati, quelli che non riguardano informazioni sul viso.

In [25]:
from mediapipe.python.solutions.pose import PoseLandmark

# ---- indici dei landmark corpo (senza viso) ----
BODY_LANDMARKS = [
    PoseLandmark.LEFT_SHOULDER, PoseLandmark.RIGHT_SHOULDER,
    PoseLandmark.LEFT_ELBOW,    PoseLandmark.RIGHT_ELBOW,
    PoseLandmark.LEFT_WRIST,    PoseLandmark.RIGHT_WRIST,
    PoseLandmark.LEFT_HIP,      PoseLandmark.RIGHT_HIP,
    PoseLandmark.LEFT_KNEE,     PoseLandmark.RIGHT_KNEE,
    PoseLandmark.LEFT_ANKLE,    PoseLandmark.RIGHT_ANKLE,
    PoseLandmark.LEFT_HEEL,     PoseLandmark.RIGHT_HEEL,
    PoseLandmark.LEFT_FOOT_INDEX, PoseLandmark.RIGHT_FOOT_INDEX,
]
BODY_IDX = [lm.value for lm in BODY_LANDMARKS]

# ---- connessioni solo corpo (niente viso) ----
BODY_CONNECTIONS = [
    # Torso / spalle / anche --> bianco
    (PoseLandmark.LEFT_SHOULDER,  PoseLandmark.RIGHT_SHOULDER,  (255, 255, 255)),
    (PoseLandmark.LEFT_SHOULDER,  PoseLandmark.LEFT_HIP,        (255, 255, 255)),
    (PoseLandmark.RIGHT_SHOULDER, PoseLandmark.RIGHT_HIP,       (255, 255, 255)),
    (PoseLandmark.LEFT_HIP,       PoseLandmark.RIGHT_HIP,       (255, 255, 255)),

    # Braccio sinistro --> blu
    (PoseLandmark.LEFT_SHOULDER, PoseLandmark.LEFT_ELBOW, (255, 0, 0)),
    (PoseLandmark.LEFT_ELBOW,    PoseLandmark.LEFT_WRIST, (255, 0, 0)),

    # Braccio destro --> rosso
    (PoseLandmark.RIGHT_SHOULDER, PoseLandmark.RIGHT_ELBOW, (0, 0, 255)),
    (PoseLandmark.RIGHT_ELBOW,    PoseLandmark.RIGHT_WRIST, (0, 0, 255)),

    # Gamba sinistra --> verde
    (PoseLandmark.LEFT_HIP,   PoseLandmark.LEFT_KNEE,       (0, 255, 0)),
    (PoseLandmark.LEFT_KNEE,  PoseLandmark.LEFT_ANKLE,      (0, 255, 0)),
    (PoseLandmark.LEFT_ANKLE, PoseLandmark.LEFT_HEEL,       (0, 255, 0)),
    (PoseLandmark.LEFT_HEEL,  PoseLandmark.LEFT_FOOT_INDEX, (0, 255, 0)),

    # Gamba destra --> giallo
    (PoseLandmark.RIGHT_HIP,   PoseLandmark.RIGHT_KNEE,       (255, 255, 0)),
    (PoseLandmark.RIGHT_KNEE,  PoseLandmark.RIGHT_ANKLE,      (255, 255, 0)),
    (PoseLandmark.RIGHT_ANKLE, PoseLandmark.RIGHT_HEEL,       (255, 255, 0)),
    (PoseLandmark.RIGHT_HEEL,  PoseLandmark.RIGHT_FOOT_INDEX, (255, 255, 0)),
]

In [26]:
def draw_body_only(frame, landmarks, img_w, img_h, visibility_thr=0.5):
    """Disegna solo i landmark e le connessioni 'corpo' con OpenCV."""
    # punti
    for idx in BODY_IDX:
        lm = landmarks[idx]
        if getattr(lm, "visibility", 1.0) < visibility_thr:
            continue
        x, y = int(lm.x * img_w), int(lm.y * img_h)
        if 0 <= x < img_w and 0 <= y < img_h:
            cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

    # linee
    for a, b, c in BODY_CONNECTIONS:
        la, lb = landmarks[a.value], landmarks[b.value]
        if getattr(la, "visibility", 1.0) < visibility_thr:
            continue
        if getattr(lb, "visibility", 1.0) < visibility_thr:
            continue
        ax, ay = int(la.x * img_w), int(la.y * img_h)
        bx, by = int(lb.x * img_w), int(lb.y * img_h)
        if (0 <= ax < img_w and 0 <= ay < img_h and
            0 <= bx < img_w and 0 <= by < img_h):
            cv2.line(frame, (ax, ay), (bx, by), c, 2)

In [28]:
from tqdm import tqdm

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_idx = 0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

with mp_pose.Pose(static_image_mode=False, model_complexity=1) as pose:
    with tqdm(total=total_frames, desc="Video elaboration", unit="frame") as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = pose.process(rgb)

            if result.pose_landmarks:
                lms = result.pose_landmarks.landmark

                # Disegna SOLO corpo
                draw_body_only(frame, lms, w, h, visibility_thr=0.5)

                # Salva SOLO corpo nel CSV (se vuoi)
                if write_csv:
                    dump_body_to_csv(lms, frame_idx, csv_writer)

            writer.write(frame)
            frame_idx += 1
            pbar.update(1)

cap.release()
writer.release()
if write_csv:
    csv_file.close()

print(f"\nVideo saved in: {out_path}")
if write_csv:
    print(f"CSV saved in: {csv_path}")

Video elaboration: 100%|██████████| 3605/3605 [02:51<00:00, 20.99frame/s]


 Video saved in: /content/drive/MyDrive/University/Deep/Sinner_2min_annoted.mp4
CSV saved in: /content/drive/MyDrive/University/Deep/Sinner_2min_landmarks.csv
